# Pystac-client Python Library
* [Medium post](https://medium.com/rotten-grapes/download-sentinel-data-within-seconds-in-python-8cc9a8c3e23c) - pystac-client & odc-stac
* [Pystac-client documentation](https://pystac-client.readthedocs.io/en/latest/)

In [96]:
import pystac_client
import pystac
import odc.stac
import rioxarray
import pathlib
import pandas
import numpy

In [2]:
def write_netcdf_conventions_in_place(
        data: xarray.core.dataarray.DataArray
    ):
        """Write the CRS and transform associated with a netCDF file such that it is CF
        complient and meets the GDAL
        expectations for transform information.

        Parameters
        ----------

        dem
            The dataset to have its spatial data written in place.
        crs_dict
            A dict with horizontal and vertical CRS information.
        """

        data.rio.write_crs(data.rio.crs, inplace=True)
        data.rio.write_transform(inplace=True)
        for key in data.data_vars:
            data[key].rio.write_crs(data[key].rio.crs, inplace=True)
            #data[key].rio.write_nodata(numpy.nan, encoded=True, inplace=True)
            compression["dtype"] = data[key].dtype
            encoding[key] = compression
        if "z" in dem:
            dem.z.rio.write_crs(crs_dict["horizontal"], inplace=True)
            dem.z.rio.write_nodata(numpy.nan, encoded=True, inplace=True)
        if "data_source" in dem:
            dem.data_source.rio.write_crs(crs_dict["horizontal"], inplace=True)
            dem.data_source.rio.write_nodata(numpy.nan, encoded=True, inplace=True)
        if "lidar_source" in dem:
            dem.lidar_source.rio.write_crs(crs_dict["horizontal"], inplace=True)
            dem.lidar_source.rio.write_nodata(numpy.nan, encoded=True, inplace=True)
        if "zo" in dem:
            dem.zo.rio.write_crs(crs_dict["horizontal"], inplace=True)
            dem.zo.rio.write_nodata(numpy.nan, encoded=True, inplace=True)

NameError: name 'xarray' is not defined

In [4]:
data_path = pathlib.Path.cwd() / ".." / "data"
(data_path / "rasters").mkdir(parents=True, exist_ok=True)
crs_wsg = 4326
crs = 2193
name = "waikouaiti"

Link to available STAC catalogues: [https://stacindex.org/catalogs#/](https://stacindex.org/catalogs#/)
* Copernicus - "https://catalogue.dataspace.copernicus.eu/stac"
* Earth Search - "https://earth-search.aws.element84.com/v1"

In [5]:
# use publically available stac link such as
odc.stac.configure_rio(cloud_defaults=True, aws={"aws_unsigned": True})
client = pystac_client.Client.open("https://earth-search.aws.element84.com/v1") 

# ID of the collection
collection = "sentinel-2-c1-l2a"

# Geometry of AOI
import geopandas
geometry_df = geopandas.read_file(data_path / "vectors" / f"{name}.gpkg")
geometry = geometry_df.to_crs(crs_wsg).iloc[0].geometry

In [27]:
# Complete month
date_YYMM = "2020-05-31"
filters = {"eo:cloud_cover":{"lt":10}} 
# run pystac client search to see available dataset
search = client.search(
    collections=[collection], intersects=geometry, datetime=date_YYMM, query=filters
) 

results_dict = search.item_collection_as_dict()
pandas.DataFrame.from_records(results_dict['features'])

,type,stac_version,id,properties,geometry,links,assets,bbox,stac_extensions,collection
0,Feature,1.0.0,S2A_T59GMK_20200531T223733_L2A,"{'created': '2024-01-22T04:29:28.342Z', 'platf...","{'type': 'Polygon', 'coordinates': [[[169.7274...","[{'rel': 'self', 'type': 'application/geo+json...",{'red': {'href': 'https://e84-earth-search-sen...,"[169.704927, -46.140291, 171.115971, -45.146216]",[https://stac-extensions.github.io/eo/v1.1.0/s...,sentinel-2-c1-l2a


Avaliable bands: ['aot', 'blue', 'coastal', 'green', 'nir', 'nir08', 'nir09', 'red', 'rededge1', 'rededge2', 'rededge3', 'scl', 'swir16', 'swir22', 'visual', 'wvp', 'aot-jp2', 'blue-jp2', 'coastal-jp2', 'green-jp2', 'nir-jp2', 'nir08-jp2', 'nir09-jp2',
 'red-jp2',
 'rededge1-jp2',
 'rededge2-jp2',
 'rededge3-jp2',
 'scl-jp2',
 'swir16-jp2',
 'swir22-jp2',
 'visual-jp2',
 'wvp-jp2']

In [127]:
bands = ["red", "green", "blue", "nir"]
#load the data in xarray format
data = odc.stac.load(search.items(), geopolygon=geometry, bands=bands,  chunks={}, ) # , groupby="solar_day" - as offsets and scale
#data

# Scale, offset, calculate NDVI, NDWI, kelp and save

In [128]:
band = 'green'
offsets = [item.assets[band].extra_fields['raster:bands'][0]['offset'] for item in search.item_collection()]
scales = [item.assets[band].extra_fields['raster:bands'][0]['scale'] for item in search.item_collection()]

In [129]:
for band in bands: 
    data[band].data = data[band].data * scales + offsets
    data[band].data[data[band].data <= 0] = numpy.nan
    data[band].rio.write_nodata(numpy.nan, encoded=True, inplace=True);

In [131]:
data["ndvi"] = (data.nir - data.red) / (data.nir + data.red)
data["ndwi"] = (data.green-data.nir)/(data.green+data.nir)
data["ndvi"].rio.write_crs(data["ndvi"].rio.crs, inplace=True)
data["ndwi"].rio.write_crs(data["ndvi"].rio.crs, inplace=True)
data["ndvi"].rio.write_nodata(numpy.nan, encoded=True, inplace=True);
data["ndwi"].rio.write_nodata(numpy.nan, encoded=True, inplace=True);

In [132]:
land = geopandas.read_file(data_path / "vectors" / "main_islands.geojson")
thresholds = {"min_ndvi": 0.01, "max_ndvi": 0.7, "max_ndwi": 0.2}

In [137]:
data["kelp"] = (data.nir - data.red) / (data.nir + data.red)
data["kelp"].data[(data["ndvi"].data < thresholds["min_ndvi"]) | (data["ndvi"].data > thresholds["max_ndvi"]) | (data["ndwi"].data > thresholds["max_ndwi"])] = numpy.nan
data["kelp"] = data["kelp"].rio.clip(land.to_crs(data["kelp"].rio.crs).geometry.values, invert=True)
data["kelp"].rio.write_crs(data["kelp"].rio.crs, inplace=True)
data["kelp"].rio.write_nodata(numpy.nan, encoded=True, inplace=True);

In [138]:
data.to_netcdf(data_path / "rasters" / f'data_with_kelp_{name}.nc', format="NETCDF4", engine="netcdf4")